# 0. Install and Import Dependencies

In [1]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python matplotlib

  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Using cached six-1.15.0-py2.py3-none-any.whl (10 kB)
  Using cached numpy-1.19.5-cp38-cp38-win_amd64.whl (13.3 MB)
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.34.1
    Uninstalling grpcio-1.34.1:
      Successfully uninstalled grpcio-1.34.1
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.0
    Uninstalling typing-extensions-3.10.0.0:
      Successfully uninstalled typing-extensions-3.10.0.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.5.0
  

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pykospacing 0.5 requires h5py==3.1.0, but you have h5py 2.10.0 which is incompatible.
pykospacing 0.5 requires tensorflow==2.5.0, but you have tensorflow 2.4.1 which is incompatible.
mxnet 1.7.0.post2 requires numpy<1.17.0,>=1.8.2, but you have numpy 1.19.5 which is incompatible.
mxnet 1.7.0.post2 requires requests<2.19.0,>=2.18.4, but you have requests 2.25.1 which is incompatible.
huggingface-hub 0.0.12 requires packaging>=20.9, but you have packaging 20.4 which is incompatible.
gensim 3.8.3 requires Cython==0.29.14, but you have cython 0.29.24 which is incompatible.
boto3 1.17.110 requires botocore<1.21.0,>=1.20.110, but you have botocore 1.18.18 which is incompatible.
awsebcli 3.20.0 requires botocore<1.21.0,>=1.19.0, but you have botocore 1.18.18 which is incompatible.
awsebcli 3.20.0 requires colorama<0.4.4,

In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

# 1. Load Model

17개의 keypoints

In [2]:
interpreter = tf.lite.Interpreter(model_path='./lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

# 2. Make Detections

keypoint ->>> [nose, left eye, right eye, left ear, right ear, left shoulder, right shoulder, left elbow, right elbow, left wrist, right wrist, left hip, right hip, left knee, right knee, left ankle, right ankle]

Input_size = 192,192,3

In [3]:
plt.imshow(tf.cast(np.squeeze(img),dtype=tf.int32))

NameError: name 'img' is not defined

In [8]:
img = frame.copy()

In [10]:
img.shape

(480, 640, 3)

In [12]:
img = tf.image.resize_with_pad(np.expand_dims(img,axis=0),192,192)


In [ ]:
img = tf.image.resize_with_pad(np.expand_dims(img,axis=0),192,192)


In [17]:
interpreter.get_input_details()

[{'name': 'serving_default_input:0',
  'index': 0,
  'shape': array([  1, 192, 192,   3]),
  'shape_signature': array([  1, 192, 192,   3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [18]:
interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 312,
  'shape': array([ 1,  1, 17,  3]),
  'shape_signature': array([ 1,  1, 17,  3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [23]:
interpreter.get_tensor(interpreter.get_output_details()[0]['index'])

array([[[[0.73403525, 0.5338897 , 0.24440986],
         [0.6947996 , 0.5626211 , 0.21794099],
         [0.7157166 , 0.5209994 , 0.23128775],
         [0.68138236, 0.5928235 , 0.17257163],
         [0.6933142 , 0.5060545 , 0.19949687],
         [0.7100148 , 0.63489866, 0.17376041],
         [0.7500121 , 0.46459314, 0.24744317],
         [0.7600616 , 0.7014748 , 0.19879764],
         [0.79286236, 0.41078818, 0.29206318],
         [0.8155749 , 0.67167914, 0.28312895],
         [0.8109863 , 0.43800104, 0.18169907],
         [0.889735  , 0.60571826, 0.29931632],
         [0.8997056 , 0.49917325, 0.2777099 ],
         [0.85034084, 0.68932295, 0.17403466],
         [0.8752548 , 0.44184646, 0.06521609],
         [0.85257477, 0.67430633, 0.15772086],
         [0.84461325, 0.515667  , 0.07451877]]]], dtype=float32)

In [9]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img,axis=0),192,192)
    input_image = tf.cast(img,dtype=tf.float32)
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])

#     print(keypoints_with_scores)
    ### keypoints - 17개
    
    # Rendering
    draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(frame, keypoints_with_scores, 0.4)
    
    
    
    cv2.imshow('Movenet Lightning', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [30]:
right_eye = keypoints_with_scores[0][0][2]
left_elbow = keypoints_with_scores[0][0][7]

In [31]:
left_elbow

array([0.96226144, 0.9714813 , 0.03983799], dtype=float32)

In [34]:
#실제 좌표
np.array(left_elbow[:2]*[480,640]).astype(int)

array([461, 621])

# 3. Draw Keypoints

In [6]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) 

# 4. Draw Edges

In [7]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [8]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)